# ETL

In [ ]:
#Importamos los paquete necesarios:
import os
import pandas as pd
import glob ##New package learned   
from sqlalchemy import create_engine

In [106]:
def extractANDclean_CSV(file_csv):
    '''Transformamos un CSV en un datafram y lo renombramos como documento procesado
    
    IN: archivo CSV
    OUT: DataFrame, csv_renamed
    '''
    print(f"Processing {file_csv} as data")
    return pd.read_csv(file_csv)

def extractANDclean_JSON(file_json):
    '''Transformamos un JSON en un datafram y lo renombramos como documento procesado

    IN: archivo JSON
    OUT: DataFrame, JSON_renamed
    '''
    print(f"Processing {file_json} as data")
    return pd.read_json(file_json, lines=True) ###REMARK

def extractANDclean_TXT(file_txt):
    '''Transformamos un TXT en un datafram y lo renombramos como documento procesado

    IN: archivo TXT
    OUT: DataFrame, TXT_renamed
    '''
    print(f"Processing {file_txt} as data")
    return pd.read_csv(file_txt,sep='|') ###REMARK

def extractANDclean_XLSX(file_xlsx):
    '''Transformamos un XLSX en un datafram y lo renombramos como documento procesado

    IN: archivo XLSX
    OUT: DataFrame, XLSX_renamed
    '''
    print(f"Processing {file_xlsx} as data")
    return pd.read_excel(file_xlsx)

In [109]:
#Damos la ruta al directorio de datos "sucios":
dirty_data = 'dirty_data/*'

df_sales = pd.DataFrame(columns=['id_venta,fecha','cliente','producto','precio','cantidad'])

#Diferenciamos por tipo de documento para poder dirigirlo a la función adecuada:
for doc in glob.glob(dirty_data): ###REMARK
    
    if doc[-4:] == ".csv": print(extractANDclean_CSV(doc).head())
    elif doc[-5:] == ".json": print(extractANDclean_JSON(doc).head())
    elif doc[-5:] == ".xlsx": print(extractANDclean_XLSX(doc).head())
        #Presuponemos que el separador para los TXT será "|":
    elif doc[-4:] == ".txt": print(extractANDclean_TXT(doc).head())
    
    #Levantamos un error si el formato no existe:
    else: 
        raise TypeError(f"Documento con formato no contemplado: {doc}")

Processing dirty_data/dataset_ventas.csv as data
   id_venta       fecha   cliente producto precio  cantidad
0         1  2023.09.26   hicham     Mouse    -10       NaN
1         2  2023.12.27     MARTA  Teclado  ERROR       NaN
2         3  2023-05-05      luis   Cables   1200       5.0
3         4         NaN    PeDro   Teclado  ERROR       5.0
4         5  2023.08.15     sofia    Mouse   1200       2.0
Processing dirty_data/dataset_ventas.json as data
   id_venta       fecha   cliente producto precio cantidad
0         1  2023.09.26   hicham     Mouse    -10     None
1         2  2023.12.27     MARTA  Teclado  ERROR     None
2         3  2023-05-05      luis   Cables   1200        5
3         4                PeDro   Teclado  ERROR        5
4         5  2023.08.15     sofia    Mouse   1200        2
Processing dirty_data/dataset_ventas.txt as data
   id_venta       fecha   cliente producto precio  cantidad
0         1  2023.09.26   hicham     Mouse    -10       NaN
1         2  2023.